In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
x=df.drop('label',axis=1)

In [6]:
 y=df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [7]:
print(x.shape)
print(y.shape)

(18285, 4)
(18285,)


In [8]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [10]:
voc_size=5000

In [11]:
messages=x.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [15]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1709, 2946, 2000, 4598, 4433, 4193, 2978, 2224, 4992, 4878],
 [4392, 1580, 4559, 2727, 4276, 3962, 690],
 [2362, 2562, 844, 3594],
 [3796, 4895, 3581, 1036, 4149, 4688],
 [1300, 4276, 4353, 230, 4963, 1139, 4276, 209, 692, 3829],
 [3339,
  472,
  1031,
  534,
  1298,
  4740,
  1423,
  4739,
  2624,
  1066,
  3621,
  3784,
  1547,
  1337,
  690],
 [2093, 2938, 3485, 577, 4638, 4105, 3551, 4390, 2535, 3281, 2012],
 [2361, 389, 2026, 87, 1200, 189, 4740, 3287, 2535, 3281, 2012],
 [4747, 208, 2438, 3943, 3377, 2448, 4642, 3065, 4740, 2287],
 [907, 4394, 865, 837, 395, 920, 1410, 4886],
 [222, 3294, 1503, 3709, 556, 4013, 1326, 3709, 1613, 1580, 498],
 [1036, 4394, 4433, 2448, 4740, 1200],
 [2888, 3804, 3183, 1378, 2279, 86, 2434, 4410, 3793],
 [4698, 452, 595, 224, 4102, 4229, 2489, 2535, 3281, 2012],
 [3748, 2257, 3173, 4003, 32, 2535, 3281, 2012],
 [101, 3407, 2275, 3056, 4300, 4192, 287, 3233, 1268, 1939],
 [1550, 4598, 1580],
 [3777, 2591, 4260, 419, 4740, 2841, 1147, 690],
 [487, 40

In [17]:
sent_length=25
embed_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embed_doc

array([[   0,    0,    0, ..., 2224, 4992, 4878],
       [   0,    0,    0, ..., 4276, 3962,  690],
       [   0,    0,    0, ..., 2562,  844, 3594],
       ...,
       [   0,    0,    0, ..., 2535, 3281, 2012],
       [   0,    0,    0, ..., 4238, 2900, 3626],
       [   0,    0,    0, ...,  754, 3590, 1627]])

In [18]:
from tensorflow.keras.layers import Dropout

In [19]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
import numpy as np
x_final=np.array(embed_doc)
y_final=np.array(y)

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33, random_state=42)

In [22]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 24ms/step - loss: 0.3584 - accuracy: 0.8260 - val_loss: 0.2063 - val_accuracy: 0.9135
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1524 - accuracy: 0.9404 - val_loss: 0.1915 - val_accuracy: 0.9234
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1116 - accuracy: 0.9586 - val_loss: 0.2097 - val_accuracy: 0.9133
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0828 - accuracy: 0.9722 - val_loss: 0.2463 - val_accuracy: 0.9186
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0665 - accuracy: 0.9787 - val_loss: 0.2378 - val_accuracy: 0.9155
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0510 - accuracy: 0.9824 - val_loss: 0.2963 - val_accuracy: 0.9162
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0383 - accuracy: 0.9876 - val_loss: 0.3361 - val_accuracy: 0.9130

In [23]:
predict_x=model.predict(x_test) 
pred=np.round(predict_x).astype(int)

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print("LSTM Data")
score=metrics.accuracy_score(y_test, pred)
print("accuracy: %0.3f" %score)
cm=metrics.confusion_matrix(y_test,pred)
cm

LSTM Data
accuracy: 0.910


array([[3144,  275],
       [ 269, 2347]], dtype=int64)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [26]:
from sklearn.metrics import precision_recall_curve
precision,recall,thres = precision_recall_curve(y_test,pred)
tpr=cm[0][0]/(cm[0][0]+cm[1][0])
fpr=cm[0][1]/(cm[0][1]+cm[1][1])
print("tpr:",tpr)
print("fpr:",fpr)

tpr: 0.9211837093466159
fpr: 0.10488176964149504
